In [215]:
import requests
from bs4 import BeautifulSoup as BS
import json

In [216]:
# list of urls
d = {}
urls = [
        'https://en.wikipedia.org/wiki/Amazon_(company)', 
        'https://en.wikipedia.org/wiki/Alibaba_Group', 
        'https://en.wikipedia.org/wiki/EBay',
        'https://en.wikipedia.org/wiki/Walmart'
       ]


# layout of wiki page is standardized
# text content is all in p tags
bottom_info_data = {}
for u in urls:
    
    # dictionary to to hold text information by url
    # in case we try and crawl all on a central notebook
    d[u] = []
    wiki_pg = requests.get(u)
    soup = BS(wiki_pg.content, 'lxml')
    for content in soup.select("p"):
        # adding to list
        d[u].append(content.text)
    
    #grabbing bottom box
    


    #hard to get just top level TR in bottom info box
    bottom_info_table = soup.select(".navbox .expanded.nowraplinks.mw-collapsible.navbox-inner > tbody > tr")

    if len(bottom_info_table) == 0:
        bottom_info_table = soup.select(".navbox .nowraplinks.mw-collapsible.navbox-inner > tbody > tr")
        
    top_level_header = bottom_info_table[0].text.replace('vte', '')
    bottom_info_data[ top_level_header] = {}
    for tr in bottom_info_table[1:]:
        header = tr.select_one("th")
        #exception is ebays weird structure, may need to add more if encounter
        if u == 'https://en.wikipedia.org/wiki/EBay':
            bottom_info_data["Ebay"] = { top_level_header: []}
            del bottom_info_data[top_level_header]
            bottom_info_data["Ebay"][ top_level_header] = [x.text.replace("\n", "(", 1).replace("\n", ",") for x in tr.select("td > div > ul > li")]
            break
            #some dont have headers
        if header:

            if "vte" in header.text:
                break
            bottom_info_data[ top_level_header][header.text] = {}
            
            #these only seem to go two level deep from what i see, if it's more than 2 , prob need to make it recursive
            isNested = tr.select_one("table")
# 
            if not isNested:
                #json structure 
                bottom_info_data[ top_level_header][header.text] = [x.text.replace("\n", "(", 1).replace("\n", ",") for x in tr.select("td > div > ul > li")]
            
            if isNested:
                nested_tr = tr.select('tr')
                
                for ntr in nested_tr:
                    nested_header = ntr.select_one("th").text
                    bottom_info_data[top_level_header][header.text][nested_header] = [x.text.replace("\n", "(", 1).replace("\n", ",") for x in ntr.select("td > div > ul > li")]
       

#print to file
with open('bottominfobox.txt', 'w') as outfile:
    json.dump(bottom_info_data, outfile)

